<a href="https://colab.research.google.com/github/Xessen/TeacherForcingLSTM/blob/main/TeacherForcingLSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import sqlite3
from tensorflow.keras.layers import Embedding,LSTM,Dense,Dropout,Input,TextVectorization
from tensorflow.keras.models import  Model
from tensorflow.keras.utils import plot_model
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import string
import nltk
from google.colab import drive


In [ ]:
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='')
tf.config.experimental_connect_to_cluster(resolver)
# This is the TPU initialization code that has to be at the beginning.
tf.tpu.experimental.initialize_tpu_system(resolver)
print("All devices: ", tf.config.list_logical_devices('TPU'))
strategy = tf.distribute.TPUStrategy(resolver)

In [ ]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip -q glove.6B.zip
drive.mount('/content/gdrive')
!cp '/content/gdrive/MyDrive/biletler/comments_data (5).db' '/content/'
drive.flush_and_unmount()

In [ ]:
conn=sqlite3.connect('/content/comments_data (5).db')
sql_q=pd.read_sql_query("SELECT parent_data,data FROM comments",conn)
df=pd.DataFrame(sql_q,columns=['parent_data','data'])
df['parent_data']=df['parent_data'].str.replace('[{}]'.format(string.punctuation), '')
df['data']=df['data'].str.replace('[{}]'.format(string.punctuation), '')

In [ ]:
vectorizer_=TextVectorization(max_tokens=8000,output_sequence_length=550)



vectorizer_.adapt(df['parent_data'])

voc_ = vectorizer_.get_vocabulary()
voc_.append('specialstartkey')
voc_.append('specialstopkey')

vectorizer=TextVectorization(max_tokens=8002,output_sequence_length=550,vocabulary=voc_)
voc=vectorizer.get_vocabulary()
word_index = dict(zip(voc, range(len(voc))))


In [ ]:

embedding_index={}
f=open("/content/glove.6B.50d.txt","r")
for line in f:
  word,coefs=line.split(maxsplit=1)
  coefs=np.fromstring(coefs,"f",sep=" ")
  embedding_index[word]=coefs


In [ ]:
num_tokens = len(voc) + 2
embedding_dim = 50


embedding_matrix = np.zeros((num_tokens, embedding_dim))
for word, i in word_index.items():
    embedding_vector = embedding_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [ ]:
from tensorflow.python.ops.numpy_ops import np_config
#with strategy.scope():
class TeacherForcingNLP(Model):
  def __init__(self,num_tokens,embedding_dim,embedding_matrix,max_len):
    super(TeacherForcingNLP,self).__init__()
    np_config.enable_numpy_behavior()

    self.encoder_input=Input(shape=(max_len,))
    self.e_embed_layer=Embedding(num_tokens,embedding_dim,embeddings_initializer=tf.keras.initializers.Constant(embedding_matrix),trainable=False,mask_zero=True)
    self.encoder_lstm=LSTM(256,return_state=True,return_sequences=True)


    self.encoder_lstm1=LSTM(256,return_state=True)


    self.decoder_input=Input(shape=(None,))
    self.d_embed_layer=Embedding(num_tokens,embedding_dim,embeddings_initializer=tf.keras.initializers.Constant(embedding_matrix),name="decoder_embed",trainable=False,mask_zero=True)

    self.decoder_lstm=LSTM(256,return_state=True,return_sequences=True,name="decoder_lstm")


    self.decoder_lstm1=LSTM(256,return_state=True,return_sequences=True,name="decoder_lstm1")

    self.decoder_dense=Dense(num_tokens,activation="softmax",name='decoder_dense')





  def call(self,inputs):
    encoder_embed_output=self.e_embed_layer(inputs[0])

    LSTM_o,state_h,state_c=self.encoder_lstm(encoder_embed_output)
    encoder_states=[state_h,state_c]
    LSTM_o1,state_h1,state_c1=self.encoder_lstm1(LSTM_o,initial_state=encoder_states)
    encoder_states_1=[state_h1,state_c1]
    decoder_embed_output=self.d_embed_layer(inputs[1])

    decoder_outputs,d_state_h,d_state_c=self.decoder_lstm(decoder_embed_output,initial_state=encoder_states_1)
    decoder_states=[d_state_h,d_state_c]

    decoder_outputs1,_,_=self.decoder_lstm1(decoder_outputs,initial_state=decoder_states)

    decoder_output=self.decoder_dense(decoder_outputs1)




    return decoder_output
  def build_graph(self):
    x = Input(shape=(550,))
    y=Input(shape=(550,))
    return Model(inputs=[x,y], outputs=self.call([x,y]))
  def inference_encoder(self):
    #einp=Input(shape=(550,))

    encoder_embed_output=self.e_embed_layer(self.encoder_input)

    LSTM_o,state_h,state_c=self.encoder_lstm(encoder_embed_output)
    encoder_states=[state_h,state_c]

    LSTM_o1,state_h1,state_c1=self.encoder_lstm1(LSTM_o,initial_state=[encoder_states[0],encoder_states[1]])
    encoder_states_1=[state_h1,state_c1]
    encoder_model=Model(self.encoder_input,encoder_states_1)



    return encoder_model

  def inference_decoder(self):

    dinp=Input(shape=(1,))

    decoder_state_input_h = Input(shape=(256,))
    decoder_state_input_c = Input(shape=(256,))
    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

    i_decoder_embed_output=self.d_embed_layer(dinp)

    i_decoder_outputs, i_state_h, i_state_c = self.decoder_lstm(
    i_decoder_embed_output, initial_state=decoder_states_inputs)
    i_decoder_states = [i_state_h, i_state_c]

    i_decoder_outputs1,i_state_h1,i_state_c1=self.decoder_lstm1(i_decoder_outputs,initial_state=i_decoder_states)
    i_decoder_states1=[i_state_h1,i_state_c1]
    
    i_dec_out=self.decoder_dense(i_decoder_outputs1)


    

    decoder_model=Model([dinp,decoder_states_inputs],[i_dec_out,i_decoder_states1])
    return decoder_model
  def inference(self,input_seq):

    encoder_model=self.inference_encoder()
    initial_states=encoder_model.predict(input_seq.reshape(1,550))
    initial_states=tf.convert_to_tensor(initial_states[0]),tf.convert_to_tensor(initial_states[1])
    

    stop_condition=False
    dec_out=8000
    decoded_seq=[]
    decoder_model=self.inference_decoder()

    while not stop_condition:
      dec_out,initial_states=decoder_model.predict([np.array(dec_out).reshape(1,1),initial_states])
      dec_out=np.argmax(dec_out[0,0,:])
      decoded_seq.append(dec_out)
      if len(decoded_seq)>25 or (dec_out==8001 and len(decoded_seq)>5):
        stop_condition=True
    return decoded_seq





In [ ]:
#with strategy.scope():
max_len=550
a_model=TeacherForcingNLP(8002,50,[],max_len)
a_model.compile(optimizer="adam",loss='sparse_categorical_crossentropy', metrics=['accuracy'])
es = tf.keras.callbacks.EarlyStopping(monitor='loss', mode='min', verbose=1, patience=2)

In [ ]:
#with strategy.scope():
X_train, X_test, y_train, y_test=train_test_split(df['parent_data'],df['data'],test_size=0.2)
x_encoder_in=vectorizer(X_train)
y_train='specialstartkey '+y_train+' specialstopkey'
x_decoder_in=vectorizer(y_train)
y_decoder_out=vectorizer(y_train)
  

In [ ]:
#with strategy.scope():
a_model.fit([x_encoder_in[200:300],x_decoder_in[200:300]],y_decoder_out[200:300],batch_size=64,epochs=1,callbacks=[es])

2/2 [==============================] - 20s 425ms/step - loss: 0.7069 - accuracy: 0.0437
